In [1]:
import os
from keras.models import load_model
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout,Convolution2D,MaxPooling2D,Flatten
from keras.optimizers import Adam


# number of classes
nb_classes = 43

# input image dimensions
img_rows, img_cols, img_ch = 32, 32, 3

# number of convolutional filters to use
nb_filter1 = 32
nb_filter2 = 64
nb_filter3 = 128



model = Sequential()
model.add(Convolution2D(32, kernel_size=(5, 5),padding='same',
                        input_shape=(img_rows,img_cols,img_ch),name='conv1'))
model.add(Activation('relu',name='relu1'))
model.add(MaxPooling2D(pool_size=(2,2),strides=1,name='maxpool1'))
model.add(Convolution2D(64, kernel_size=(5, 5),padding='same',
                        name='conv2'))
model.add(Activation('relu',name='relu2'))
model.add(MaxPooling2D(pool_size=(2,2),strides=None,name='maxpool2'))
model.add(Convolution2D(128, kernel_size=(5, 5), padding='same',
                        name='conv3'))
model.add(Activation('relu',name='relu3'))
model.add(MaxPooling2D(pool_size=(2,2),strides=None,name='maxpool3'))
model.add(Flatten(name='flatten'))
model.add(Dropout(0.5,name='dropout1'))
model.add(Dense(128, name='hidden1'))
model.add(Activation('relu',name='relu4'))
model.add(Dropout(0.5,name='dropout2'))
model.add(Dense(128,  name='hidden2'))
model.add(Activation('relu',name='relu5'))
model.add(Dense(nb_classes, name='output'))
model.add(Activation('softmax',name='softmax'))

model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1 (Conv2D)               (None, 32, 32, 32)        2432      
_________________________________________________________________
relu1 (Activation)           (None, 32, 32, 32)        0         
_________________________________________________________________
maxpool1 (MaxPooling2D)      (None, 31, 31, 32)        0         
_________________________________________________________________
conv2 (Conv2D)               (None, 31, 31, 64)        51264     
_________________________________________________________________
relu2 (Activation)           (None, 31, 31, 64)        0         
_________________________________________________________________
maxpool2 (MaxPooling2D)      (None, 15, 15, 64)        0         
_________________________________________________________________
conv3 (Conv2D)               (None, 15, 15, 128)       204928    
__________

In [4]:
import pickle

training_file = 'train.p'
testing_file = 'test.p'
valid_file = 'valid.p'
 
with open(training_file, mode='rb') as f:
    train = pickle.load(f)
with open(testing_file, mode='rb') as f:
    test = pickle.load(f)
with open(valid_file, mode='rb') as f:
    valid = pickle.load(f)


In [5]:
    X_train, y_train, X_test, y_test = train['features'], train['labels'],test['features'], test['labels']
    X_valid, y_valid = test['features'], test['labels']